In [1]:
# /content/drive/MyDrive/age.zip
!mkdir dataset

In [2]:
!unzip /content/drive/MyDrive/age.zip -d dataset

Archive:  /content/drive/MyDrive/age.zip
   creating: dataset/fifties/
  inflating: dataset/fifties/fifties_actor-001.jpeg  
  inflating: dataset/fifties/fifties_actor-002.jpeg  
  inflating: dataset/fifties/fifties_actor-003.jpeg  
  inflating: dataset/fifties/fifties_actor-004.jpeg  
  inflating: dataset/fifties/fifties_actor-005.jpeg  
  inflating: dataset/fifties/fifties_actor-006.jpeg  
  inflating: dataset/fifties/fifties_actor-007.jpeg  
  inflating: dataset/fifties/fifties_actor-008.jpeg  
  inflating: dataset/fifties/fifties_actor-009.jpeg  
  inflating: dataset/fifties/fifties_actor-010.jpeg  
  inflating: dataset/fifties/fifties_actor-011.jpeg  
  inflating: dataset/fifties/fifties_actor-012.jpeg  
  inflating: dataset/fifties/fifties_actor-013.jpeg  
  inflating: dataset/fifties/fifties_actor-014.jpeg  
  inflating: dataset/fifties/fifties_actor-015.jpeg  
  inflating: dataset/fifties/fifties_actor-016.jpeg  
  inflating: dataset/fifties/fifties_actor-017.jpeg  
  inflating

In [9]:
from google.colab import drive
import PIL # python 의 이미지 전처리 라이브러리
import glob # 대용량, 대규모 파일을 다룰 때 사용
import os # os, path 관련 조작, 변경, 파일 삭제 등
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

dataset_folder_path = '/content/dataset'
category_cnt = os.listdir(dataset_folder_path)
os.listdir(dataset_folder_path)


['fifties', 'twenties']

In [4]:
# glob.glob(dataset_folder_path+'/*/*')
# 파일의 경로+/* 을 입력하면 해당 폴더 하위 경로 상에 있는 모든 파일을
# 리스트로 리턴
# ex) dataset_folder_path+'/*/*' -> 하위 폴더의 하위 데이터 전체 리턴
import numpy as np
x = []
y = []
for label, folder in enumerate(os.listdir(dataset_folder_path)):
   # print(dataset_folder_path + '/'+ folder) 좋은 코드가 아님
   # print(os.path.join(dataset_folder_path, folder)) # 유지 보수 좋은 코드
    folder_full_path = os.path.join(dataset_folder_path, folder)
    all_files = glob.glob(folder_full_path + '/*.jpeg')

    for idx, file_full_path in enumerate(all_files):
        image = PIL.Image.open(file_full_path)
        image = image.resize((64, 64))
        image = image.convert('RGB')
        data = np.asarray(image)
        x.append(data)
        y.append(label)
        if idx % 30 == 0:
            print(idx, '/', len(all_files))

0 / 358
30 / 358
60 / 358
90 / 358
120 / 358
150 / 358
180 / 358
210 / 358
240 / 358
270 / 358
300 / 358
330 / 358
0 / 212
30 / 212
60 / 212
90 / 212
120 / 212
150 / 212
180 / 212
210 / 212


In [5]:
x = np.array(x)
y = np.array(y)
print('카테고리 갯수', np.bincount(y))
print('전처리 확인', x.shape)

카테고리 갯수 [358 212]
전처리 확인 (570, 64, 64, 3)


In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
# 개발 팁, 전처리 시간이 너무 오래 걸린다면(데이터 늘어 나서)
# xy = (x_train, x_test, y_train, y_test)
# np.save('/content/drive/MyDrive/preprocessing_data.npy', xy)
# x_train, x_test, y_train, y_test = np.load('/content/drive/MyDrive/preprocessing_data.npy')

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [7]:
x_train = x_train / 255.
x_test = x_test / 255.

In [ ]:
# 구글넷 실습 (오전에 진행한)
# 레즈넷 실습 (다 짜여진 코드) -> finetuning

In [10]:
# 개선의 여지 해상도 수정 ( 전처리 도 수정 해야 함)
input_layer = layers.Input(shape = (64,64,3), name='start_layer')

# 개선의 여지 conv2d 수정 성능이 매우 잘 나온다면 -모델 다이어트
# 성능이 안나온다면 channel 수, 레이어 추가
tower_1 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)

tower_2 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)
tower_2 = layers.Conv2D(64, kernel_size = (3,3) , padding='same' , activation= 'relu')(tower_2)

tower_3 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)
tower_3 = layers.Conv2D(64, kernel_size = (5,5) , padding='same' , activation= 'relu')(tower_3)

tower_4 = layers.MaxPool2D(pool_size=(3,3), strides=(1,1) , padding='same')(input_layer)
tower_4 = layers.Conv2D(64, kernel_size=(1,1) , padding='same' , activation='relu')(tower_4)

concat_layer = layers.concatenate([tower_1, tower_2,tower_3,tower_4], axis=3)

tower_2_1 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)

tower_2_2 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)
tower_2_2 = layers.Conv2D(120, kernel_size = (3,3) , padding='same' , activation= 'relu')(tower_2_2)

tower_2_3 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)
tower_2_3 = layers.Conv2D(120, kernel_size = (5,5) , padding='same' , activation= 'relu')(tower_2_3)

tower_2_4 = layers.MaxPool2D(pool_size=(3,3), strides=(1,1) , padding='same')(concat_layer)
tower_2_4 = layers.Conv2D(120, kernel_size=(1,1) , padding='same' , activation='relu')(tower_2_4)

final_concat_layer = layers.concatenate([tower_2_1, tower_2_2,tower_2_3,tower_2_4], axis=3)
average_pooling_layer = layers.AveragePooling2D(pool_size=(16,16),strides=(16,16))(final_concat_layer)
flat_layer = layers.Flatten()(average_pooling_layer)

# 개선의 여지 -  Dense 층 추가
output_layer = layers.Dense(1 , activation = 'sigmoid')(flat_layer)

google_net_model = models.Model(inputs = input_layer, outputs = output_layer)
google_net_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 start_layer (InputLayer)    [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 64, 64, 64)           256       ['start_layer[0][0]']         
                                                                                                  
 conv2d_3 (Conv2D)           (None, 64, 64, 64)           256       ['start_layer[0][0]']         
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 64, 64, 3)            0         ['start_layer[0][0]']         
 D)                                                                                           

In [11]:
google_net_model.compile(loss='binary_crossentropy',
                         optimizer='adam', metrics=['accuracy'])

In [12]:
check_point_path = '/content/drive/MyDrive/ourmodel_checkpoint'
mc = tf.keras.callbacks.ModelCheckpoint(check_point_path,
                                   monitor='val_loss',
                                   save_best_only=True)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

In [13]:
google_net_model.fit(x_train,y_train, batch_size=32,
                     epochs=20,
                     callbacks=[mc,es],
                     validation_data=(x_test,y_test))

Epoch 1/20
15/15 [==============================] - 282s 19s/step - loss: 0.6832 - accuracy: 0.5855 - val_loss: 0.6631 - val_accuracy: 0.6491
Epoch 2/20
15/15 [==============================] - 280s 19s/step - loss: 0.6562 - accuracy: 0.6228 - val_loss: 0.6416 - val_accuracy: 0.6404
Epoch 3/20
15/15 [==============================] - 269s 18s/step - loss: 0.6262 - accuracy: 0.6579 - val_loss: 0.7326 - val_accuracy: 0.6404
Epoch 4/20
15/15 [==============================] - 259s 17s/step - loss: 0.6430 - accuracy: 0.6272 - val_loss: 0.6254 - val_accuracy: 0.6667
Epoch 5/20
15/15 [==============================] - 262s 18s/step - loss: 0.6043 - accuracy: 0.6776 - val_loss: 0.6344 - val_accuracy: 0.6842
Epoch 6/20
15/15 [==============================] - 265s 18s/step - loss: 0.6021 - accuracy: 0.6930 - val_loss: 0.6138 - val_accuracy: 0.7018
Epoch 7/20
15/15 [==============================] - 267s 18s/step - loss: 0.5796 - accuracy: 0.7018 - val_loss: 0.5933 - val_accuracy: 0.6842
Epoch 

In [14]:
## our model inference code
# inference만 진행 할 시, 가장 위 import library 실행 -> 모델 빌드 하는 부분 실행
google_net_model.load_weights('/content/drive/MyDrive/ourmodel_checkpoint')

In [15]:
from google.colab import files
f = files.upload()

Saving twenties_actor-10.jpeg to twenties_actor-10.jpeg


In [16]:
file_name = list(f.keys())[0]
test_img = PIL.Image.open(file_name)
test_img = test_img.resize((64,64))
test_img = test_img.convert('RGB')
data = np.asarray(test_img)
test_image_tensor = np.array([data])
test_image_tensor = test_image_tensor / 255.
test_image_tensor.shape

(1, 64, 64, 3)

In [17]:
result = google_net_model.predict(test_image_tensor)
result[0][0]


1/1 [==============================] - 0s 316ms/step


0.2842254

In [18]:
if result[0][0] > 0.5:
    print(f'{result[0][0] * 100} % 확률로 50대.')
else:
    print(f'{ (1 - result[0][0]) * 100} % 확률로 20대.')

71.57745957374573 % 확률로 20대.


In [ ]:
# 1. 데이터
# 2. 모델
# 3. 하이퍼 파라미터 수정 (우선 순위 최하)